In [ ]:
# ! pip install catboost
# ! pip install seaborn
# ! pip install imbalanced-learn

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import random
from tqdm import tqdm as tqdm


# -------------------------------------------------
import lightgbm as lgbm
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import Pool, CatBoostClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GroupShuffleSplit

# ---------------------------------------------------
from sklearn.cluster import KMeans

# from featexp import get_univariate_plots
from imblearn.under_sampling import ClusterCentroids
from imblearn.under_sampling import TomekLinks
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer

# -------------------------data-----------------------#
import requests
from urllib.parse import urlparse
from pathlib import Path
from datetime import datetime

# Required libraries
import tifffile as tiff
import datetime
import matplotlib.pyplot as plt

import rasterio
from rasterio.plot import show
from rasterio.windows import Window
from rasterio.enums import Resampling
from rasterio.transform import Affine

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Path to the input GeoTIFF file
input_file = r"D:\projects\kelp_wanted\data\train_satellite\AA498489_satellite.tif"
mask_file = r"D:\projects\kelp_wanted\data\train_kelp\AA498489_kelp.tif"

In [ ]:
train_satellite_path = r"D:\projects\kelp_wanted\data\train_satellite"
train_label_path = r"D:\projects\kelp_wanted\data\train_kelp"

In [ ]:
# Get the list of train files
train_input = os.listdir(train_satellite_path)
train_input_id = [d.split("_")[0] for d in train_input if d.endswith(".tif")]

# Get the list of train labels
train_label = os.listdir(train_label_path)

In [ ]:
train_input_id

In [ ]:
# Define scale factor and offset
scale_factor = 0.0000275
offset = -0.2

In [ ]:
# # make a dataframe with column for 7 bands, label and id
# df = pd.DataFrame(columns=["B01", "B02", "B03", "B04", "B05", "B06", "B07", "label", "id"])

In [ ]:
# train_input_id[:10]

In [ ]:
dfs = []  # Initialize list to store DataFrames
# count = 0

# Loop over each file
for file in tqdm(train_input_id):  # Adjust the range as needed
    # Read the GeoTIFF file
    with rasterio.open(train_satellite_path + "\\" + file + "_satellite.tif") as src:
        # Read all bands
        band_data = src.read().astype(np.float32)  # Convert to float32
        # Get the shape of the image
        height, width = src.height, src.width
        # Apply scale factor and offset to all bands except band 6 and band 7
        for i in range(7):
            if i != 5 and i != 6:  # Skip band 6 and band 7
                band_data[i] = band_data[i] * scale_factor + offset

        # Create a mask where band 7 values are greater than 0
        land_mask = band_data[6] > 0
        # Apply the land mask to all bands except band 6 and 7
        for i in range(5):
            band_data[i][land_mask] = 0

        # Find the indices of pixels not in the land mask
        non_land_mask_indices = np.where(~land_mask)

        # Read the binary mask TIFF file
        with rasterio.open(train_label_path + "\\" + file + "_kelp.tif") as mask_src:
            # Read the mask data
            kelp_mask = mask_src.read(1)
            # Find the locations where the mask values are 1
            kelp_mask_indices = np.where(kelp_mask == 1)
            
        # """ Select all non land pixels
        # Filter pixels with band 6 values greater than 0
        selected_indices_filtered = (
            non_land_mask_indices[0][band_data[5][non_land_mask_indices] == 0],
            non_land_mask_indices[1][band_data[5][non_land_mask_indices] == 0],
        )
        # """

        '''# Select a random sample of pixels
        num_random_samples = min(
            len(non_land_mask_indices[0]), len(kelp_mask_indices[0]), 1000
        )
        random_indices = random.sample(
            range(len(non_land_mask_indices[0])), num_random_samples
        )
        # Combine indices of kelp mask pixels and random sample pixels
        selected_indices = (
            np.concatenate(
                (kelp_mask_indices[0], non_land_mask_indices[0][random_indices])
            ),
            np.concatenate(
                (kelp_mask_indices[1], non_land_mask_indices[1][random_indices])
            ),
        )

        # Filter pixels with band 6 values greater than 0
        selected_indices_filtered = (
            selected_indices[0][band_data[5][selected_indices] == 0],
            selected_indices[1][band_data[5][selected_indices] == 0],
        )
        # '''

        # Extract band values for each selected pixel
        pixel_values = band_data[
            :, selected_indices_filtered[0], selected_indices_filtered[1]
        ].T

        # Create a DataFrame for the selected pixels
        df = pd.DataFrame(pixel_values, columns=[f"B0{i+1}" for i in range(7)])

        # Create an array of labels
        labels = np.zeros(len(selected_indices_filtered[0]))
        labels[: len(kelp_mask_indices[0])] = 1  # Set labels to 1 for kelp mask pixels
        df["label"] = labels
        df["id"] = file
        # count = count + 1
        # df["count"] = count

        # Append DataFrame to the list
        dfs.append(df)

# Concatenate all DataFrames in the list
final_df = pd.concat(dfs, ignore_index=True)

# Shuffle the DataFrame rows
# df_shuffled = final_df.sample(frac=1, random_state=42).reset_index(drop=True)

df_shuffled = pd.DataFrame(final_df)

In [ ]:
np.where(df_shuffled.label == 0)[0].shape

In [ ]:
len(final_df["id"].unique())

In [ ]:
final_df

In [ ]:
band_data[6].shape

In [ ]:
# # Read the GeoTIFF file
# with rasterio.open(input_file) as src:
#     # Read all bands
#     band_data = src.read().astype(np.float32)  # Convert to float32
#     # Get the shape of the image
#     height, width = src.height, src.width
#     # Apply scale factor and offset to all bands except band 6 and band 7
#     for i in range(7):
#         if i != 5 and i != 6:  # Skip band 6 and band 7
#             band_data[i] = band_data[i] * scale_factor + offset
            
#     # Create a mask where band 6 values are greater than 0
#     land_mask = band_data[6] > 0
#     # Apply the land mask to all bands except band 6
#     for i in range(7):
#         band_data[i][land_mask] = 0

#     # Find the indices of pixels not in the land mask
#     non_land_mask_indices = np.where(~land_mask)
    
#     # Read the binary mask TIFF file
#     with rasterio.open(mask_file) as mask_src:
#         # Read the mask data
#         kelp_mask = mask_src.read(1)
#         # Find the locations where the mask values are 1
#         kelp_mask_indices = np.where(kelp_mask == 1)

#     # Select a random sample of pixels
#     num_random_samples = min(
#         len(non_land_mask_indices[0]), len(kelp_mask_indices[0]), 1000
#     )
#     random_indices = random.sample(
#         range(len(non_land_mask_indices[0])), num_random_samples
#     )
#     # Combine indices of kelp mask pixels and random sample pixels
#     selected_indices = (
#         np.concatenate(
#             (kelp_mask_indices[0], non_land_mask_indices[0][random_indices])
#         ),
#         np.concatenate(
#             (kelp_mask_indices[1], non_land_mask_indices[1][random_indices])
#         ),
#     )

#     # Create a dictionary to store band values for each selected pixel
#     data_dict = {f"B0{i+1}": band_data[i][selected_indices] for i in range(7)}
#     # Create an array of labels
#     labels = np.zeros(len(selected_indices[0]))
#     labels[:len(kelp_mask_indices[0])] = 1  # Set labels to 1 for kelp mask pixels
    
#     # Add labels as a new column to the DataFrame
#     data_dict["label"] = labels
#     data_dict["id"] = input_id

#     # Create a DataFrame from the dictionary
#     df = pd.DataFrame(data_dict)
#     # Shuffle the DataFrame rows
#     df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
df_shuffled

In [ ]:
# Calculate NDVI
# (N - R)/(N + R)	
df_shuffled["NDVI"] = (df_shuffled["B02"] - df_shuffled["B03"]) / (
    df_shuffled["B02"] + df_shuffled["B03"]
)

In [ ]:
# Calculate NDWI
# (N - S1) / (N + S1)
df_shuffled["NDWI"] = (df_shuffled["B02"] - df_shuffled["B01"]) / (
    df_shuffled["B02"] + df_shuffled["B01"]
)

In [ ]:
# Calculate G-NDVI
# (N - G) / (N + G)
df_shuffled["G-NDVI"] = (df_shuffled["B02"] - df_shuffled["B04"]) / (
    df_shuffled["B02"] + df_shuffled["B04"]
)

In [ ]:
# Calculate GRVI 
# (N / G)
df_shuffled["GRVI"] = (df_shuffled["B02"] / df_shuffled["B04"])

In [ ]:
# # TGI	Triangular Greenness Index	-(-0.5*((R-B)*(R-G)-(R-G)*(R-B)))
# df_shuffled["TGI"] = -(-0.5 * (
#     (df_shuffled["B03"] - df_shuffled["B05"]) * (df_shuffled["B03"] - df_shuffled["B04"]) 
#     - 
#     (df_shuffled["B03"] - df_shuffled["B04"]) * (df_shuffled["B03"] - df_shuffled["B05"])
#     )
# )

In [ ]:
# TVI	Triangular Vegetation Index	0.5 * (120 * (N - G) - 200 * (R - G))
df_shuffled["TVI"] = 0.5 * (120 * (df_shuffled["B02"] - df_shuffled["B04"]) - 200 * (df_shuffled["B03"] - df_shuffled["B04"]))

In [ ]:
# # Calculate NIRv
# # ((N - R) / (N + R)) * N
# df_shuffled["NIRv"] = ((df_shuffled["B02"] - df_shuffled["B03"]) / (df_shuffled["B02"] + df_shuffled["B03"])) * df_shuffled["B03"]

In [ ]:
# # CIgreen(Chlorophyll Index Green)
# # (N / G) - 1.0
# df_shuffled["CIgreen"] = (df_shuffled["B02"] / df_shuffled["B04"])

In [ ]:
# # Find count of label 1 and 0
#     # label_counts = df_shuffled['label'].value_counts()


#     # Path to save the filtered GeoTIFF file
#     output_file = (
#         r"D:\projects\kelp_wanted\data\test_data\AA498489_satellite_filtered.tif"
#     )

#     # Calculate the new transformation matrix with a vertical flip
#     transform = Affine(
#         src.transform.a, 0, src.transform.c, 0, -src.transform.e, src.transform.f
#     )

#     # Write the filtered image to a new GeoTIFF file
#     with rasterio.open(
#         output_file,
#         "w",
#         driver="GTiff",
#         width=width,
#         height=height,
#         count=7,
#         dtype=band_data.dtype,
#         crs=src.crs,
#         transform=transform,
#     ) as dst:
#         for i in range(7):
#             dst.write(band_data[i], i + 1)

# print("Filtered image saved as:", output_file)

In [ ]:
df_shuffled.head(10)

In [ ]:
df_shuffled.to_csv(r"D:\projects\kelp_wanted\data\data_full.csv", index=False)

# Train/Test Split

### method 1

In [ ]:
# Split the dataset into features (X) and labels (y)
X = df_shuffled.drop(
    columns=["id", "label", "B06", "B07"]
)  # Features excluding 'id' and 'label' columns
y = df_shuffled["label"]  # Target variable

# Specify groups based on the 'id' column
groups = df_shuffled["id"]

# Initialize the GroupShuffleSplit with a specified test size
group_splitter = GroupShuffleSplit(test_size=0.2, random_state=42)

# Split the data into train and test sets while preserving group membership
train_idx, test_idx = next(group_splitter.split(X, y, groups=groups))

# Split the data into train and test sets
X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

### Train and Save the model

In [ ]:
# Initialize and train the CatBoostClassifier
model = CatBoostClassifier(
    task_type="GPU",
    iterations=5000,
    learning_rate=0.001,
    random_strength=0.1,
    depth=8,
    loss_function="MultiClass",
)
model.fit(X_train.fillna(0), y_train)

# Define the file path where you want to save the model
model_file_path = r"D:\projects\kelp_wanted\model\catboost_model_4.bin"
# Save the model
model.save_model(model_file_path)

### method 2

In [ ]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load your dataset (assuming it's already loaded into X and y)

# Split the dataset into features (X) and labels (y)
X = df_shuffled.drop(
    columns=["id", "label", "B06", "B07"]
)  # Features excluding 'id' and 'label' columns
y = df_shuffled["label"]  # Target variable

# Split the data into train and test sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

# Create CatBoost Pool objects for training and validation data
train_pool = Pool(data=X_train, label=y_train)
val_pool = Pool(data=X_val, label=y_val)

# Initialize and train the CatBoostClassifier
model = CatBoostClassifier(
    task_type="GPU",
    iterations=5000,
    learning_rate=0.001,
    random_strength=0.1,
    depth=8,
    loss_function="MultiClass",
    l2_leaf_reg=3,  # L2 regularization strength
    subsample=0.8,  # Subsample ratio for training
    verbose=100,  # Print every 100 iterations
)
model.fit(
    train_pool,
    eval_set=val_pool,  # Use a validation set to monitor performance
    early_stopping_rounds=100,  # Stop training if the performance doesn't improve for 100 iterations
    plot=True,  # Plot the training metrics during training
)

# Get predictions on the validation set
val_preds = model.predict(X_val)

# Evaluate accuracy
val_accuracy = accuracy_score(y_val, val_preds)
print("Validation Accuracy:", val_accuracy)

# Get feature importance
feature_importance = model.get_feature_importance(train_pool)
print("Feature Importance:", feature_importance)

### Load the Model

In [ ]:
# model 5 path
model_v5_path = r"D:\projects\kelp_wanted\model\catboost_model_5_new_method.bin"

In [ ]:
# Load the saved model
model_v5 = CatBoostClassifier()
model_v5.load_model(model_v5_path)

### Test the model

In [ ]:
# Get predicted probabilities
preds = loaded_model.predict_proba(X_test.fillna(0))

### Generate result

In [ ]:
# Convert predicted probabilities to class predictions
threshold = 0.95  # Adjust threshold as needed
pred_labels = (preds[:, 1] > threshold).astype(int)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, pred_labels)

# Compute total number of samples
total_samples = len(y_test)

# Convert confusion matrix to percentage values
conf_matrix_percentage = (conf_matrix / total_samples) * 100

print("Confusion Matrix (Percentage):")
print(conf_matrix_percentage)

In [ ]:
# Compute accuracy
accuracy = accuracy_score(y_test, pred_labels)

# Compute precision
precision = precision_score(y_test, pred_labels)

# Compute recall
recall = recall_score(y_test, pred_labels)

# Compute F1-score
f1 = f1_score(y_test, pred_labels)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

In [ ]:
# Generate confusion matrix
conf_matrix = confusion_matrix(y_test, pred_labels)

# Plot heatmap
plt.figure(figsize=(5, 5))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted labels")
plt.ylabel("True labels")
plt.title("Confusion Matrix")
plt.show()

# TEST

In [ ]:
# def calculate_bands(df):
#     # Calculate NDVI
#     # (N - R)/(N + R)
#     df["NDVI"] = (df["B02"] - df["B03"]) / (
#         df["B02"] + df["B03"]
#     )
#     # Calculate NDWI
#     # (N - S1) / (N + S1)
#     df["NDWI"] = (df["B02"] - df["B01"]) / (
#         df["B02"] + df["B01"]
#     )
#     # Calculate G-NDVI
#     # (N - G) / (N + G)
#     df["G-NDVI"] = (df["B02"] - df["B04"]) / (
#         df["B02"] + df["B04"]
#     )
#     # Calculate GRVI
#     # (N / G)
#     df["GRVI"] = df["B02"] / df["B04"]
#     # # TGI	Triangular Greenness Index	-(-0.5*((R-B)*(R-G)-(R-G)*(R-B)))
#     # df["TGI"] = -(-0.5 * (
#     #     (df["B03"] - df["B05"]) * (df["B03"] - df["B04"])
#     #     -
#     #     (df["B03"] - df["B04"]) * (df["B03"] - df["B05"])
#     #     )
#     # )
#     # TVI	Triangular Vegetation Index	0.5 * (120 * (N - G) - 200 * (R - G))
#     df["TVI"] = 0.5 * (
#         120 * (df["B02"] - df["B04"])
#         - 200 * (df["B03"] - df["B04"])
#     )
    
#     return df

In [ ]:
test_satellite_path = r"D:\projects\kelp_wanted\data\test_satellite"

In [ ]:
# Get the list of train files
test_satellite = os.listdir(test_satellite_path)
test_satellite_id = [d.split("_")[0] for d in test_satellite if d.endswith(".tif")]

In [ ]:
test_satellite_id[:4]

In [ ]:
for file in tqdm(test_satellite_id):
    # Read the id_kelp.tif file
    with rasterio.open(test_satellite_path + "\\" + file + "_satellite.tif") as kelp_src:
        # with rasterio.open(test_file) as kelp_src:
        kelp_mask = kelp_src.read(1)
        band_data = kelp_src.read().astype(np.float32)  # Convert to float32

    # Create a zero image of the same dimension
    zero_image = np.zeros_like(kelp_mask)

    # """
    # Read all bands
    # Get the shape of the image
    height, width = kelp_src.height, kelp_src.width
    # Apply scale factor and offset to all bands except band 6 and band 7
    for i in range(5):
        band_data[i] = band_data[i] * scale_factor + offset

    # Create a land mask
    land_mask = band_data[6] > 0  # Assuming band 6 represents land
    # """
    # Read non-land pixels from the input image
    non_land_pixels = band_data[:, ~land_mask]

    # Remember the positions of non-land pixels
    non_land_indices = np.where(~land_mask)

    # Extract band values for each selected pixel
    pixel_values = band_data[:, non_land_indices[0], non_land_indices[1]].T

    # Create a DataFrame for the selected pixels
    df = pd.DataFrame(pixel_values, columns=[f"B0{i+1}" for i in range(7)])

    # dfs.append(df)
    # # Concatenate all DataFrames in the list
    # test_final_df = pd.concat(df, ignore_index=True)
    # Shuffle the DataFrame rows
    # df = df.sample(frac=1, random_state=42).reset_index(drop=True)

    # Calculate NDVI
    # (N - R)/(N + R)
    df["NDVI"] = (df["B02"] - df["B03"]) / (
        df["B02"] + df["B03"]
    )
    # Calculate NDWI
    # (N - S1) / (N + S1)
    df["NDWI"] = (df["B02"] - df["B01"]) / (
        df["B02"] + df["B01"]
    )
    # Calculate G-NDVI
    # (N - G) / (N + G)
    df["G-NDVI"] = (df["B02"] - df["B04"]) / (
        df["B02"] + df["B04"]
    )
    # Calculate GRVI
    # (N / G)
    df["GRVI"] = df["B02"] / df["B04"]
    # # TGI	Triangular Greenness Index	-(-0.5*((R-B)*(R-G)-(R-G)*(R-B)))
    # df["TGI"] = -(-0.5 * (
    #     (df["B03"] - df["B05"]) * (df["B03"] - df["B04"])
    #     -
    #     (df["B03"] - df["B04"]) * (df["B03"] - df["B05"])
    #     )
    # )
    # TVI	Triangular Vegetation Index	0.5 * (120 * (N - G) - 200 * (R - G))
    df["TVI"] = 0.5 * (
        120 * (df["B02"] - df["B04"])
        - 200 * (df["B03"] - df["B04"])
    )

    # new_df = calculate_bands(df)

    df_shuffled = df.drop(columns=["B06", "B07"])

    # Get predicted probabilities
    test_preds_v5 = model_v5.predict_proba(df_shuffled.fillna(0))

    # Convert predicted probabilities to class predictions
    threshold = 0.9  # Adjust threshold as needed
    test_pred_labels = (test_preds_v5[:, 1] > threshold).astype(int)

    # Plant predictions on the zero image
    zero_image[non_land_indices] = test_pred_labels

    # Generate the binary result
    binary_result = np.where(zero_image > 0, 1, 0)

    # Save res
    # Write the binary image to a GeoTIFF file
    output_binary_image = f"D:\\projects\\kelp_wanted\\data\\test_tiff_data\\{file.split('_')[0]}_kelp.tif"
    # output_binary_image = (
    #     r"D:\projects\kelp_wanted\data\test_tiff_data\JS569579_kelp_90.tif"
    # )

    with rasterio.open(
        output_binary_image,
        "w",
        driver="GTiff",
        height=height,
        width=width,
        count=1,  # Single band
        dtype=binary_result.dtype,
        crs=kelp_src.crs,
        # transform=src.transform,
    ) as dst:
        # Write the final binary image to the GeoTIFF file
        dst.write(binary_result, 1)

# Single Image test

In [ ]:
test_file = r"D:\projects\kelp_wanted\data\train_satellite\JS569579_satellite.tif"  

# Read the id_kelp.tif file
with rasterio.open(test_file) as kelp_src:
    kelp_mask = kelp_src.read(1)
    band_data = kelp_src.read().astype(np.float32)  # Convert to float32

# Create a zero image of the same dimension
zero_image = np.zeros_like(kelp_mask)

# """
# Read all bands
# Get the shape of the image
height, width = kelp_src.height, kelp_src.width
# Apply scale factor and offset to all bands except band 6 and band 7
for i in range(5):
    band_data[i] = band_data[i] * scale_factor + offset

# Create a land mask
land_mask = band_data[6] > 0  # Assuming band 6 represents land
# """
# Read non-land pixels from the input image
non_land_pixels = band_data[:, ~land_mask]

# Remember the positions of non-land pixels
non_land_indices = np.where(~land_mask)

# Extract band values for each selected pixel
pixel_values = band_data[:, non_land_indices[0], non_land_indices[1]].T

# Create a DataFrame for the selected pixels
df = pd.DataFrame(pixel_values, columns=[f"B0{i+1}" for i in range(7)])

# dfs.append(df)

# # Concatenate all DataFrames in the list
# test_final_df = pd.concat(df, ignore_index=True)

# # Shuffle the DataFrame rows
# test_df_shuffled = test_final_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
df

In [ ]:
# Calculate NDVI
# (N - R)/(N + R)
df["NDVI"] = (df["B02"] - df["B03"]) / (
    df["B02"] + df["B03"]
)
# Calculate NDWI
# (N - S1) / (N + S1)
df["NDWI"] = (df["B02"] - df["B01"]) / (
    df["B02"] + df["B01"]
)
# Calculate G-NDVI
# (N - G) / (N + G)
df["G-NDVI"] = (df["B02"] - df["B04"]) / (
    df["B02"] + df["B04"]
)
# Calculate GRVI
# (N / G)
df["GRVI"] = df["B02"] / df["B04"]
# # TGI	Triangular Greenness Index	-(-0.5*((R-B)*(R-G)-(R-G)*(R-B)))
# df["TGI"] = -(-0.5 * (
#     (df["B03"] - df["B05"]) * (df["B03"] - df["B04"])
#     -
#     (df["B03"] - df["B04"]) * (df["B03"] - df["B05"])
#     )
# )
# TVI	Triangular Vegetation Index	0.5 * (120 * (N - G) - 200 * (R - G))
df["TVI"] = 0.5 * (
    120 * (df["B02"] - df["B04"])
    - 200 * (df["B03"] - df["B04"])
)

In [ ]:
df

In [ ]:
# Get predicted probabilities
test_preds = model_v5.predict_proba(df)

# Convert predicted probabilities to class predictions
threshold = 0.97  # Adjust threshold as needed
test_pred_labels = (test_preds[:, 1] > threshold).astype(int)

In [ ]:
(np.where(test_pred_labels == 1))[0].shape

In [ ]:
# Plant predictions on the zero image
zero_image[non_land_indices] = test_pred_labels

# Generate the binary result
binary_result = np.where(zero_image > 0, 1, 0)

In [ ]:
binary_result.shape

In [ ]:
# Write the binary image to a GeoTIFF file
output_binary_image = r"D:\projects\kelp_wanted\data\test_data\JS569579_kelp_97_v5.tif"

# # Calculate the new transformation matrix with a vertical flip
# transform = Affine(
#     test_src.transform.a,
#     0,
#     test_src.transform.c,
#     0,
#     -test_src.transform.e,
#     test_src.transform.f,
# )

with rasterio.open(
    output_binary_image,
    "w",
    driver="GTiff",
    height=height,
    width=width,
    count=1,  # Single band
    dtype=binary_result.dtype,
    crs=src.crs,
    # transform=src.transform,
) as dst:
    # Write the final binary image to the GeoTIFF file
    dst.write(binary_result, 1)

In [ ]:
# # Read the GeoTIFF file
# with rasterio.open(test_file) as test_src:
#     # Read all bands
#     band_data = test_src.read().astype(np.float32)  # Convert to float32
#     # Get the shape of the image
#     height, width = test_src.height, test_src.width
#     # Apply scale factor and offset to all bands except band 6 and band 7
#     for i in range(5):
#         band_data[i] = band_data[i] * scale_factor + offset

#     # Reshape each band's data to 1-dimensional array
#     reshaped_bands = {f"B0{i+1}": band_data[i].ravel() for i in range(5)}

#     # Create a DataFrame from the dictionary
#     test_df = pd.DataFrame(reshaped_bands)

In [ ]:
# # Get predicted probabilities
# test_preds = model.predict_proba(test_df.fillna(0))

In [ ]:
# # Convert predicted probabilities to class predictions
# threshold = 0.5  # Adjust threshold as needed
# test_pred_labels = (test_preds[:, 1] > threshold).astype(int)

In [ ]:
# import rasterio
# from rasterio.transform import from_origin
# from rasterio.enums import Resampling
# from rasterio.warp import calculate_default_transform, reproject, Resampling

# # Define output file path for the binary image
# # output_binary_image = "output_binary_image.tif"
# output_binary_image = (
#     r"D:\projects\kelp_wanted\data\test_data\AF169120_res.tif"
# )

# # Get spatial properties of the original GeoTIFF file
# transform = test_src.transform
# crs = test_src.crs
# dtype = test_src.dtypes[0]

# # Define pixel values for the two classes (0 and 1)
# class_0_value = 0
# class_1_value = 255  # Assuming 8-bit integer image

# # Reshape the predicted labels to match the original image dimensions
# test_pred_labels_reshaped = test_pred_labels.reshape(height, width)

# # Calculate the new transformation matrix with a vertical flip
# transform = Affine(
#     test_src.transform.a, 0, test_src.transform.c, 0, -test_src.transform.e, test_src.transform.f
# )

# # Write the binary image
# with rasterio.open(
#     output_binary_image,
#     "w",
#     driver="GTiff",
#     height=height,
#     width=width,
#     count=1,  # Single band
#     dtype=dtype,
#     crs=crs,
#     transform=transform,
# ) as dst:
#     # Write predicted labels to the binary image


#     dst.write(test_pred_labels_reshaped.astype(dtype), 1)

In [ ]:
# Convert predicted probabilities to class predictions
threshold = 0.5  # Adjust threshold as needed
pred_labels = (preds[:, 1] > threshold).astype(int)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_val, pred_labels)

# Compute total number of samples
total_samples = len(y_val)

# Convert confusion matrix to percentage values
conf_matrix_percentage = (conf_matrix / total_samples) * 100

print("Confusion Matrix (Percentage):")
print(conf_matrix_percentage)

In [ ]:
# Compute accuracy
accuracy = accuracy_score(y_val, pred_labels)

# Compute precision
precision = precision_score(y_val, pred_labels)

# Compute recall
recall = recall_score(y_val, pred_labels)

# Compute F1-score
f1 = f1_score(y_val, pred_labels)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

In [ ]:
# Generate confusion matrix
conf_matrix = confusion_matrix(y_val, pred_labels)

# Plot heatmap
plt.figure(figsize=(5, 5))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted labels")
plt.ylabel("True labels")
plt.title("Confusion Matrix")
plt.show()